### Import

In [ ]:
import altair as alt
from operator import itemgetter

%run implementations.ipynb
%run utilities.ipynb

### Load Data

In [ ]:
sizes = pd.read_csv('npmpackages/npm_no_scope_full_stats_nonzero_downloads.csv', header=None)
sizes.head()

In [ ]:
sizes = pd.read_csv('unsplash/lite/unsplash_lite.csv', header=None)
sizes.head()

### AUC

In [ ]:
c_list = [1.0, 1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07, 1.08, 1.09, 1.1]
conf_list = [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10,
                  0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.20,
                  0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.30,
                  0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.40,
                  0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.50,
                  0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.60,
                  0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.70,
                  0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.80,
                  0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.90,
                  0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0]

In [ ]:
size_list = sizes[1].tolist()

size_counts = defaultdict(int)

for size in size_list:
    size_counts[size] += 1
    
uniq_sizes = sorted(list(size_counts.keys()))

In [ ]:
with open('jsvuln/jsvuln.txt') as f:
#with open('nature/nature_urls.txt') as f:
    vuln_objects = set(f.read().splitlines())
    
total_downloads = sizes[2].sum()

vuln_list = []

for row in sizes.itertuples(index=False):
    if row[0] in vuln_objects:
        vuln_list.append((row[0], row[1], row[2]/total_downloads))

In [ ]:
AUC = []

In [ ]:
for c in c_list:
    i_max, cndl_idxs, p_ji, p_i, s, p_j_dict, size_to_i, this_MI, max_L = BlahutArimoto_init(sizes, c, max_itr=10000000, eps=1e-2)

    for v_id, v_size, v_prob in vuln_list:
        this_i = size_to_i[v_size]
        i_num_cndls = i_max[this_i] - this_i + 1
        i_idx = cndl_idxs[this_i]
        
        recall_prec_list = []
        
        for conf in conf_list:
            TP, TN, FP, FN = 0, 0, 0, 0            
        
            for offset in range(i_num_cndls):
                this_j = this_i + offset
                j_size = s[this_j]
                idx = i_idx + offset
                prob = p_ji[idx]
                
                total = p_j_dict[j_size]
                
                positive = prob*v_prob
                negative = total - positive
            
                if positive / total >= conf:
                    TP += positive
                    FP += negative
                else:
                    TN += negative
                    FN += positive                    
    
            if TP + FP > 0 and TP + FN > 0:
                recall = TP / (TP + FN)
                prec = TP / (TP + FP) 
                recall_prec_list.append((recall, prec))
                
        recall_prec_list.sort(key=itemgetter(0))
        
        recall_prec_list = [(0.0, recall_prec_list[0][1])] + recall_prec_list
        
        df_recall_prec = pd.DataFrame(recall_prec_list, columns =['x', 'y'])
        auc = np.trapz(df_recall_prec['y'].tolist(), x=df_recall_prec['x'].tolist())
        
        if auc > 1.0:
            auc = 1.0
            
        AUC.append(("\\perReqAlg", c, v_id, auc))        
                
    print(c)

In [ ]:
for c in c_list:
    i_max, cndl_idxs, p_ji, p_i, s, p_j_dict, size_to_i, this_MI, max_L = p_alpaca_return_all(sizes, c)

    for v_id, v_size, v_prob in vuln_list:
        this_i = size_to_i[v_size]
        i_num_cndls = i_max[this_i] - this_i + 1
        i_idx = cndl_idxs[this_i]
        
        recall_prec_list = []
        
        for conf in conf_list:
            TP, TN, FP, FN = 0, 0, 0, 0            
        
            for offset in range(i_num_cndls):
                this_j = this_i + offset
                j_size = s[this_j]
                idx = i_idx + offset
                prob = p_ji[idx]
                
                total = p_j_dict[j_size]
                
                positive = prob*v_prob
                negative = total - positive
            
                if positive / total >= conf:
                    TP += positive
                    FP += negative
                else:
                    TN += negative
                    FN += positive                    
    
            if TP + FP > 0 and TP + FN > 0:
                recall = TP / (TP + FN)
                prec = TP / (TP + FP) 
                recall_prec_list.append((recall, prec))
                
        recall_prec_list.sort(key=itemgetter(0))
        
        recall_prec_list = [(0.0, recall_prec_list[0][1])] + recall_prec_list
        
        df_recall_prec = pd.DataFrame(recall_prec_list, columns =['x', 'y'])
        auc = np.trapz(df_recall_prec['y'].tolist(), x=df_recall_prec['x'].tolist())
        
        if auc > 1.0:
            auc = 1.0
            
        AUC.append(("\\pAlpaca", c, v_id, auc))        
                
    print(c)

In [ ]:
for c in c_list:
    partition, dyn_solution = opt_partition_MI(sizes, c)

    size_map = {}    
    current_block = 0
    current_ceiling = partition[current_block][1]

    for size in uniq_sizes:
        if size > current_ceiling:
            current_block += 1
            current_ceiling = partition[current_block][1]
        size_map[size] = current_ceiling

    p_j_dict = defaultdict(float)

    for row in sizes.itertuples(index=False):
        p_j_dict[size_map[row[1]]] += row[2]/total_downloads
    
    for v_id, v_size, v_prob in vuln_list:
        j_size = size_map[v_size]
        
        recall_prec_list = []
        
        for conf in conf_list:
            TP, TN, FP, FN = 0, 0, 0, 0            
        
            total = p_j_dict[j_size]
                
            positive = v_prob
            negative = total - positive
            
            if positive / total >= conf:
                TP += positive
                FP += negative
            else:
                TN += negative
                FN += positive                    
    
            if TP + FP > 0 and TP + FN > 0:
                recall = TP / (TP + FN)
                prec = TP / (TP + FP) 
                recall_prec_list.append((recall, prec))
                
        recall_prec_list.sort(key=itemgetter(0))
        
        recall_prec_list = [(0.0, recall_prec_list[0][1])] + recall_prec_list
        
        df_recall_prec = pd.DataFrame(recall_prec_list, columns =['x', 'y'])
        auc = np.trapz(df_recall_prec['y'].tolist(), x=df_recall_prec['x'].tolist())
        
        if auc > 1.0:
            auc = 1.0
            
        AUC.append(("\\perObjAlg", c, v_id, auc))        
        
    print(c)

In [ ]:
for c in c_list:
    size_map = pureMaxL(sizes, c)

    p_j_dict = defaultdict(float)

    for row in sizes.itertuples(index=False):
        p_j_dict[size_map[row[1]]] += row[2]/total_downloads
    
    for v_id, v_size, v_prob in vuln_list:
        j_size = size_map[v_size]
        
        recall_prec_list = []
        
        for conf in conf_list:
            TP, TN, FP, FN = 0, 0, 0, 0            
        
            total = p_j_dict[j_size]
                
            positive = v_prob
            negative = total - positive
            
            if positive / total >= conf:
                TP += positive
                FP += negative
            else:
                TN += negative
                FN += positive                    
    
            if TP + FP > 0 and TP + FN > 0:
                recall = TP / (TP + FN)
                prec = TP / (TP + FP) 
                recall_prec_list.append((recall, prec))
                
        recall_prec_list.sort(key=itemgetter(0))
        
        recall_prec_list = [(0.0, recall_prec_list[0][1])] + recall_prec_list
        
        df_recall_prec = pd.DataFrame(recall_prec_list, columns =['x', 'y'])
        auc = np.trapz(df_recall_prec['y'].tolist(), x=df_recall_prec['x'].tolist())
        
        if auc > 1.0:
            auc = 1.0
            
        AUC.append(("\\noDistAlg", c, v_id, auc))        
              
    print(c)

In [ ]:
for c in c_list:
    min_size = sizes[1].min()
    bin_size = int(c*min_size) - min_size
    
    p_j_dict = defaultdict(float)

    for row in sizes.itertuples(index=False):
        p_j_dict[getDALPaCA(row[1], bin_size)] += row[2]/total_downloads
    
    for v_id, v_size, v_prob in vuln_list:
        j_size = getDALPaCA(v_size, bin_size)
        
        recall_prec_list = []
        
        for conf in conf_list:
            TP, TN, FP, FN = 0, 0, 0, 0            
        
            total = p_j_dict[j_size]
                
            positive = v_prob
            negative = total - positive
            
            if positive / total >= conf:
                TP += positive
                FP += negative
            else:
                TN += negative
                FN += positive                    
    
            if TP + FP > 0 and TP + FN > 0:
                recall = TP / (TP + FN)
                prec = TP / (TP + FP) 
                recall_prec_list.append((recall, prec))
                
        recall_prec_list.sort(key=itemgetter(0))
        
        recall_prec_list = [(0.0, recall_prec_list[0][1])] + recall_prec_list
        
        df_recall_prec = pd.DataFrame(recall_prec_list, columns =['x', 'y'])
        auc = np.trapz(df_recall_prec['y'].tolist(), x=df_recall_prec['x'].tolist())
        
        if auc > 1.0:
            auc = 1.0
            
        AUC.append(("\\dAlpaca", c, v_id, auc))        
        
    print(c)

In [ ]:
for c in [1.09]:
    p_j_dict = defaultdict(float)

    for row in sizes.itertuples(index=False):
        p_j_dict[getPadme(row[1])] += row[2]/total_downloads
    
    for v_id, v_size, v_prob in vuln_list:
        j_size = getPadme(v_size)
        
        recall_prec_list = []
        
        for conf in conf_list:
            TP, TN, FP, FN = 0, 0, 0, 0            
        
            total = p_j_dict[j_size]
                
            positive = v_prob
            negative = total - positive
            
            if positive / total >= conf:
                TP += positive
                FP += negative
            else:
                TN += negative
                FN += positive                    
    
            if TP + FP > 0 and TP + FN > 0:
                recall = TP / (TP + FN)
                prec = TP / (TP + FP) 
                recall_prec_list.append((recall, prec))
                
        recall_prec_list.sort(key=itemgetter(0))
        
        recall_prec_list = [(0.0, recall_prec_list[0][1])] + recall_prec_list
        
        df_recall_prec = pd.DataFrame(recall_prec_list, columns =['x', 'y'])
        auc = np.trapz(df_recall_prec['y'].tolist(), x=df_recall_prec['x'].tolist())
        
        if auc > 1.0:
            auc = 1.0
            
        AUC.append(("\\padme", c, v_id, auc))        
        
    print(c)

In [ ]:
df_AUC = pd.DataFrame(AUC, columns =['Method', 'c', 'v_id', 'auc'])

In [ ]:
df_AUC_temp = df_AUC.loc[df_AUC['c'] == 1.09, ['Method', 'v_id', 'auc']]

alt.Chart(df_AUC_temp).transform_density(
    'auc',
    as_=['AUC', 'density'],
    extent=[0, 1],
    groupby=['Method']
).mark_area(orient='horizontal').encode(
    #y='AUC:Q',
    y=alt.Y(
        'AUC:Q',
        axis=alt.Axis(title=""),
    ),
    #color='Method:N',
    #color=alt.Color('Method:N', legend=None),
    x=alt.X(
        'density:Q',
        stack='center',
        impute=None,
        title=None,
        axis=alt.Axis(labels=False, values=[0],grid=False, ticks=True),
    ),
    column=alt.Column(
        'Method:N',
        sort=["\\perReqAlg", "\\perObjAlg", "\\noDistAlg", "\\pAlpaca", "\\dAlpaca", "\\padme"],
        title="",
        header=alt.Header(
            titleOrient='bottom',
            labelOrient='bottom',
            labelPadding=0,
        ),
    )
).properties(
    width=100,
    height=150
).configure_facet(
    spacing=0
).configure_view(
    stroke=None
)

In [ ]:
df_AUC.to_csv('evaluation/auc-recall-precision-nodejs.csv')

__Save SVG for__ $\LaTeX{}$

In [ ]:
df_AUC = pd.read_csv('evaluation/auc-recall-precision-unsplash.csv')

In [ ]:
df_AUC_temp = df_AUC.loc[df_AUC['c'] == 1.03, ['Method', 'v_id', 'auc']]

alt.Chart(df_AUC_temp).transform_density(
    'auc',
    as_=['AUC', 'density'],
    extent=[0, 1],
    groupby=['Method']
).mark_area(orient='horizontal').encode(
    #y='AUC:Q',
    y=alt.Y(
        'AUC:Q',
        axis=alt.Axis(title=""),
    ),
    #color='Method:N',
    #color=alt.Color('Method:N', legend=None),
    x=alt.X(
        'density:Q',
        stack='center',
        impute=None,
        title=None,
        axis=alt.Axis(labels=False, values=[0],grid=False, ticks=True),
    ),
    column=alt.Column(
        'Method:N',
        sort=["\\perReqAlg", "\\perObjAlg", "\\noDistAlg", "\\pAlpaca", "\\dAlpaca", "\\padme"],
        title="",
        header=alt.Header(
            titleOrient='bottom',
            labelOrient='bottom',
            labelPadding=0,
        ),
    )
).properties(
    width=75,
    height=150
).configure_facet(
    spacing=0
).configure_view(
    stroke=None
)